### Import 

In [1]:
import pandas as pd
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
import re
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load Data

In [5]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("../raw_data/okcupid_profiles.csv")

In [4]:
df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,68.0,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,has cats,NaN,pisces but it doesn&rsquo;t matter,no,"english, french, c++","i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,NaN,pisces,no,"english, german (poorly)",i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",66.0,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,likes dogs and likes cats,NaN,aquarius,no,english,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


In [ ]:
df.pets.value_counts()

In [ ]:
df.pets=df.pets.fillna('likes dogs and likes cats')

df.pets.replace(['likes dogs','likes dogs and has cats','has dogs','has dogs and likes cats','has dogs and has cats','has cats','likes cats'],'likes dogs and likes cats',inplace=True)

df.pets.replace(['has dogs and dislikes cats','dislikes cats'],'likes dogs and dislikes cats',inplace=True)

df.pets.replace(['dislikes dogs and has cats','dislikes dogs'],'dislikes dogs and likes cats',inplace=True)

In [ ]:
df.head()

### My topics to clean

essay8- The most private thing I am willing to admit

essay9- You should message me if...

status, diet, ethnicity, last_online, religion

we have decided to remove last_online

#### status

In [ ]:
df.status.replace("unknown","available",inplace=True)

In [ ]:
df['status'] = np.where(df['status'].str.contains('single'), 'available', df['status'])
df = pd.concat([pd.get_dummies(df.status, prefix='status', prefix_sep='_'), df], axis = 1)

df.status.value_counts()

In [ ]:
df.head(30)

In [ ]:
df['status'] = df['status'].map({'unknown': 'available', 'male': 0})

#### diet

In [ ]:
df.diet.fillna("no restriction", inplace=True)

# create dummy variable: strict (1=strictly following a diet)
df['strict'] = 0
df.loc[df.diet.str.contains('strictly'), 'strict'] = 1
df.loc[df.diet.str.len()==1, 'strict'] = 1

# group diets
df['diet'] = np.where(df['diet'].str.contains('strictly anything|mostly other|anything|mostly anything|strictly other|other'), 'no restriction', df['diet'])
df.loc[df.diet=='no restriction', 'strict'] = 0
df['diet'] = np.where(df['diet'].str.contains('mostly vegetarian|strictly vegan|strictly vegetarian|mostly vegan|vegan|vegetarian'), 'veggie', df['diet'])
df['diet'] = np.where(df['diet'].str.contains('mostly kosher|strictly kosher|kosher'), 'kosher', df['diet'])
df['diet'] = np.where(df['diet'].str.contains('mostly halal|strictly halal|halal'), 'halal', df['diet'])

In [ ]:
df[df["status"]=="unknown"]

In [ ]:
df[df["diet"]=="veggie"]

In [ ]:
df.diet.unique()

#### ethnicity

We can split ethnicity into 6 big groups

white,asian,hispanic/latin,black,middle eastern

In [ ]:
df['eth_num'] = df.ethnicity.str.len()
df["ethnicity2"] = "race"
df.loc[df.eth_num<2, "ethnicity2"] = df.ethnicity.str[0]
df.loc[df.eth_num==2, "ethnicity2"] = "biracial"
df.loc[df.eth_num>2, "ethnicity2"] = "multiracial"
df.loc[df.eth_num>2, "eth_num"] = 3

In [ ]:
df.head()

#### religion

In [ ]:
(df.religion.isnull().sum()/len(df))*100

In [ ]:
df["religion"] = df["religion"].fillna('agnostic')

In [ ]:
df.religion.isnull().sum()

In [ ]:
pd.set_option('display.max_columns', None)

##### we will have to try different methods of fill NA in our model
such as fill NA as agnostic or other

#### we can also try to split the data in different ways such as
shrink into 6 categories or 
split into categories and add column for intesity 


In [ ]:
df.religion.value_counts()

##### fill NA with agnostic



In [ ]:
dfagn = df.copy()

dfagn["religion"] = df["religion"].fillna("agnosticism")

##### fill NA with other

In [ ]:
dfother = df.copy()
dfother["religion"] = df["religion"].fillna("other")

##### add features for seriousness

In [ ]:
dfagn['rel_scale'] = np.where(dfagn['religion'].str.contains("very serious"), 1, 0)

In [ ]:
def create_religion_scale(df):
    conditions = [
        df['religion'].str.contains("very serious"),
        df['religion'].str.contains("somewhat serious"),
        df['religion'].str.contains("not too serious"),
        df['religion'].str.contains("laughing")]
    choices = ['very serious', 'somewhat serious', 'not too serious','laughing']
    df['rel_scale'] = np.select(conditions, choices, default='normal')
    return df

In [ ]:
df.religion.fillna("other", inplace=True)

# create 1 variable: serious (1=yes, 0=neutral, -1=no)
df["serious"] = 0
df.loc[df.religion.str.contains("very|somewhat"), "serious"] = 1
df.loc[df.religion.str.contains("laughing"), "serious"] = -1
df.religion = df.religion.str.split().str[0]
# df.groupby("religion")["serious"].mean()

# seriousness by religion
plt.plot(df.groupby("religion")["serious"].mean(), marker='o', color="#5e3a98")
plt.xticks(rotation=45)
plt.title("how seriously does a user take religion?")
plt.ylabel("average seriousness")
plt.show()


In [ ]:
dfagn = create_religion_scale(dfagn)
dfagn

In [ ]:
dfother = create_religion_scale(dfother)

dfother

#### cleaning function

In [ ]:
def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized


In [ ]:
df['essay0']=df['essay8'].fillna('')
df['essay1']=df['essay9'].fillna('')
df['essay2']=df['essay8'].fillna('')
df['essay3']=df['essay9'].fillna('')
df['essay4']=df['essay8'].fillna('')
df['essay5']=df['essay9'].fillna('')
df['essay6']=df['essay8'].fillna('')
df['essay7']=df['essay9'].fillna('')
df['essay8']=df['essay8'].fillna('')
df['essay9']=df['essay9'].fillna('')


In [ ]:
df['essay0clean'] = df['essay0'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay1clean'] = df['essay1'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay2clean'] = df['essay2'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay3clean'] = df['essay3'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay4clean'] = df['essay4'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay5clean'] = df['essay5'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay6clean'] = df['essay6'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay7clean'] = df['essay7'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay8clean'] = df['essay8'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))
df['essay9clean'] = df['essay9'].apply(lambda x: clean(x)).apply(lambda x:' '.join(x))

In [ ]:
dfessay = df[['essay0clean','essay1clean','essay2clean','essay3clean','essay4clean','essay5clean','essay6clean','essay7clean','essay8clean','essay9clean']].copy()

In [ ]:
dfessay.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5,ngram_range=(3,3))

data_vectorized = vectorizer.fit_transform(dfessay["essay0clean"])

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2,perplexity=40,random_state=42)
tsne_essays = tsne.fit_transform(data_vectorized)

In [ ]:
plt.scatter(tsn)

In [ ]:
df['essay8clean'][0]

In [ ]:
df['essay8'][0]

In [ ]:
df['essay9clean'][0]

In [ ]:
df['essay9'][0]

In [6]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [7]:
from RecommenDate import clean_data

[nltk_data] Downloading package punkt to /home/rohito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rohito/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rohito/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rohito/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
clean_df = clean_data.clean_data(df)

In [9]:
clean_df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,religion_info,strict,speaks_cleaned,primary_language,number_of_languages,essay0_cleaned,essay1_cleaned,essay2_cleaned,essay3_cleaned,essay4_cleaned,essay5_cleaned,essay6_cleaned,essay7_cleaned,essay8_cleaned,essay9_cleaned
0,22,single,m,straight,a little extra,no restriction,socially,never,NaN,"asian, white",75.0,104395,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,agnosticism,0,[english],english,1,would love think kind intellectual either dumb...,currently working international agent freight ...,making people laugh ranting good salting findi...,way look six foot half asian half caucasian mu...,book absurdistan republic mouse men book made ...,food water cell phone shelter,duality humorous thing,trying find someone hang anything except club,new california looking someone wisper secret,want swept foot tired norm want catch coffee b...
1,35,single,m,straight,average,no restriction,often,sometimes,NaN,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,,,i am very open and will share just about anyth...,,agnosticism,0,"[english, fluently, spanish, poorly, french, p...",english,4,chef mean workaholic love cook regardless whet...,dedicating everyday unbelievable badass,silly ridiculous amonts fun wherever smart as ...,,die hard christopher moore fan really watch lo...,delicious porkness glory big as doughboy sinki...,,,open share anything,
2,38,available,m,straight,thin,no restriction,socially,rather not say,NaN,NaN,68.0,104395,other,2012-06-27-09-10,"san francisco, california",rather not say,likes dogs and likes cats,agnosticism,pisces but it doesn&rsquo;t matter,no,"english, french, c++","i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",agnosticism,0,"[english, french, c]",english,3,ashamed much writing public text online dating...,make nerdy software musician artist experiment...,improvising different context alternating pres...,large jaw large glass physical thing people co...,okay cultural matrix get specific like crossha...,movement conversation creation contemplation t...,,viewing listening dancing talking drinking per...,five year old known boogerman,bright open intense silly ironic critical cari...
3,23,single,m,straight,thin,veggie,socially,rather not say,NaN,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes dogs and 

In [11]:
ls ..

Makefile     notebooks/  README.md      requirements.txt  setup.py
MANIFEST.in  raw_data/   RecommenDate/  scripts/          tests/


In [14]:
from pathlib import Path  
filepath = Path('../raw_data/clean_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
clean_df.to_csv(filepath) 